In [ ]:
"""
# For Google Colab training

!pip install pytorch_lightning pedalboard

from google.colab import drive
drive.mount('/content/drive')

from drive.MyDrive.apr.src.datasets import VoxCelebDataModule
from drive.MyDrive.apr.src.losses import (
    AAMSoftmaxLoss, SubCenterAAMSoftmaxLoss, SoftmaxLoss
)
from drive.MyDrive.apr.src.models import (
    ResNet34, build_efficientnetv2,
    Var_ECAPA, MHA_LAS, ResNet34SE, ResNet20
)
from drive.MyDrive.apr.src.resnetse import ResNetSE, SEBasicBlock, ResNetSEV2

import warnings
warnings.filterwarnings(
    "ignore", ".*Trying to infer the `batch_size`.*"
)
warnings.filterwarnings(
    "ignore", ".*The dataloader,.*"
)

import torch
from torch import nn
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

seed_everything(42, workers=True)

!unzip "/content/drive/MyDrive/datasets/voxceleb1/subset3.zip" -d "/content/subset"
"""

In [1]:
import warnings
warnings.filterwarnings(
    "ignore", ".*Trying to infer the `batch_size`.*"
)
warnings.filterwarnings(
    "ignore", ".*The dataloader,.*"
)

import torch
from torch import nn
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from src.datasets import VoxCelebDataModule
from src.losses import (
    AAMSoftmaxLoss, SubCenterAAMSoftmaxLoss, SoftmaxLoss
)
from src.models import (
    ResNet34, build_efficientnetv2,
    Var_ECAPA, MHA_LAS, ResNet34SE, ResNet20,
    ResNet34SEV2
)
from src.resnetse import ResNetSE, SEBasicBlock, ResNetSEV2

seed_everything(42, workers=True)

Global seed set to 42


42

block diagram, ntalam paper acoustic surveillance, hmm, add the number of utterances for train, val and test in stats. Recognition rate of k-means. Confusion matrix? See what mistakes we make, by taking the nationality and gender of the errors? Test with full length. Fusion? 
Slides: Models diagrams
Bigger font in k-means, also vertical. Check that in verification there are no speakers already in training
K-means, take only 10 speeches per speaker, also need to extract logmel from test. Or simply do everything on test
Recreate the dataset and test that it works...

In [2]:
num_classes = 100
batch_size = 8
n_mels = 80
from_memory = False
spec_augment = False
data_dir = "/media/gabriele/Seagate Expansion Drive/Datasets/VoxCeleb1/"
colab_data_dir = "/content/"
num_workers = 4


dm2 = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=2,
    # num_workers=num_workers,
    # data_dir=data_dir
)
dm3 = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=3,
    # num_workers=num_workers,
    # data_dir=data_dir
)
dm4 = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=4,
    clip_test=True
    # num_workers=num_workers,
    # data_dir=data_dir
)
dm6 = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=6,
    # num_workers=num_workers,
    # data_dir=data_dir
)

dmn = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=None,
    # num_workers=num_workers,
    # data_dir=data_dir
)

# ResNet34

## Train (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=aams,
    optimizer="Adam",
    lr_scheduler=None
)
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

## Test (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
checkpoint_path = "checkpoints/resnet34/last.ckpt"
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    scale=30,
    margin=0.2
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = ResNet34(num_classes=num_classes, loss_func=aams) \
        .load_from_checkpoint(
            checkpoint_path, 
            num_classes=num_classes, 
            loss_func=aams
        )
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    gradient_clip_val=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    resume_from_checkpoint=checkpoint_path
)
trainer.test(
    model, 
    dm3,
    ckpt_path=checkpoint_path
)

# ResNet34-SE

## Train (3 secs)

In [ ]:
# 64 epochs with m 0.1 and s 15

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34se/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15,
    num_subcenters=3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SE(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="AdamW",
    lr=0.001,
    # lr_scheduler="CyclicLR",
    # lr_scheduler_interval="step"
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34se",
    sub_dir="3secs",
    version=1
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    precision=16,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm3,
    ckpt_path="checkpoints/resnet34se/3secs/last.ckpt"
)

## Train (4 secs)

In [ ]:
checkpoint_dir = "/content/drive/MyDrive/checkpoints/"
logs_dir = "/content/drive/MyDrive/tb_logs"

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34se/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SE(
    embeddings_dim=256,
    num_classes=num_classes,
    n_mels=n_mels,
    loss_func=softmax,
    optimizer="Adam",
    # lr_scheduler="CyclicLR",
    # lr_scheduler_interval="step"
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34se",
    sub_dir="4secs"
)
trainer = Trainer(
    deterministic=True,
    gpus=1,
    gradient_clip_val=1,
    precision=16,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    # ckpt_path="checkpoints/resnet34se/3secs/last.ckpt"
)

## Train (6 secs)

In [ ]:
# From 80 epochs

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34se/6secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.3,
    scale=30
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SE(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr_scheduler=None
)
logger = TensorBoardLogger("tb_logs", name="resnet34se")
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model=model, 
    datamodule=dm6,
    ckpt_path="checkpoints/resnet34se/3secs/last.ckpt"
)

# ResNet34-SEV2

## Train (2 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34sev2/2secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15,
    num_subcenters=3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SEV2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr=0.001,
    # lr_scheduler="CyclicLR",
    # lr_scheduler_interval="step"
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34sev2",
    sub_dir="2secs",
    version=0
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    precision=16,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm2,
    # ckpt_path="checkpoints/resnet34sev2/3secs/last.ckpt"
)

## Train (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34sev2/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15,
    num_subcenters=3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SEV2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr=0.001,
    # lr_scheduler="CyclicLR",
    # lr_scheduler_interval="step"
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34sev2",
    sub_dir="3secs",
    version=0
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    precision=16,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm3,
    ckpt_path="checkpoints/resnet34sev2/3secs/last.ckpt"
)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34sev2/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.15,
    scale=20,
    num_subcenters=3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SEV2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr=0.001,
    # lr_scheduler="CyclicLR",
    # lr_scheduler_interval="step"
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34sev2",
    sub_dir="4secs",
    version=1
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    precision=16,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    ckpt_path="checkpoints/resnet34sev2/2secs/epoch=18-step=17574.ckpt"
)

## Test (4 secs)

In [4]:
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.15,
    scale=20,
    num_subcenters=3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SEV2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr=0.001,
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34sev2",
    sub_dir="4secs",
    version=1
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    precision=16,
    gradient_clip_val=1,
    num_sanity_val_steps=0
)
trainer.test(
    model, 
    dm4,
    ckpt_path="checkpoints/resnet34sev2/4secs/epoch=23-step=22199.ckpt"
)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\core\datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
Restoring states from the checkpoint path at checkpoints/resnet34sev2/4secs/epoch=23-step=22199.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at checkpoints/resnet34sev2/4secs/epoch=23-step=22199.ckpt


Testing:  99%|█████████▉| 83/84 [00:06<00:00, 14.78it/s]USA          0.693277
UK           0.205882
Mexico       0.025210
Chile        0.021008
Spain        0.016807
Italy        0.012605
Germany      0.008403
Australia    0.008403
Canada       0.004202
Norway       0.004202
dtype: float64
m    0.60084
f    0.39916
dtype: float64


Testing: 100%|██████████| 84/84 [00:25<00:00, 14.78it/s]

Computing scores: 100%|██████████| 10/10 [06:50<00:00, 41.03s/it]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.6421052813529968,
 'test_acc_epoch': 0.6421052813529968,
 'test_f1': 0.5928391218185425,
 'test_f1_epoch': 0.5928391218185425,
 'test_top1_acc': 0.6421052813529968,
 'test_top1_acc_epoch': 0.6421052813529968,
 'test_top5_acc': 0.8421052694320679,
 'test_top5_acc_epoch': 0.8421052694320679}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 84/84 [08:26<00:00,  6.03s/it]


D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\core\datamodule.py:469: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


[{'test_acc': 0.6421052813529968,
  'test_acc_epoch': 0.6421052813529968,
  'test_f1': 0.5928391218185425,
  'test_f1_epoch': 0.5928391218185425,
  'test_top1_acc': 0.6421052813529968,
  'test_top1_acc_epoch': 0.6421052813529968,
  'test_top5_acc': 0.8421052694320679,
  'test_top5_acc_epoch': 0.8421052694320679}]

# EfficientNetV2

## Train (3 secs)

In [ ]:
checkpoint_dir = "/content/drive/MyDrive/checkpoints/"
logs_dir = "/content/drive/MyDrive/tb_logs"

checkpoint_callback = ModelCheckpoint(
    dirpath=checkpoint_dir + "efficientnetv2/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
model = build_efficientnetv2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="Adam",
    # lr=0.001,
    #lr_scheduler="ReducePlateau",
    #lr_scheduler_interval="epoch"
)

logger = TensorBoardLogger(
    logs_dir, 
    name="efficientnetv2",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    precision=16,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm3,
    ckpt_path=checkpoint_dir + "efficientnetv2/3secs/last.ckpt"
)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/efficientnetv2/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
model = build_efficientnetv2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="Adam",
    #lr=0.001,
    #lr_scheduler="ReducePlateau",
    #lr_scheduler_interval="epoch"
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="efficientnetv2",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    precision=16,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(
    model, 
    dm4,
    ckpt_path="checkpoints/efficientnetv2/4secs/last.ckpt"
)

## Test (4 secs)

In [ ]:
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = build_efficientnetv2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="Adam"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="efficientnetv2",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    precision=16,
    num_sanity_val_steps=0
)
trainer.test(
    model, 
    dm4,
    ckpt_path="checkpoints/efficientnetv2/4secs/epoch=69-step=82305.ckpt"
)

# MHA_LAS

## Train (2 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/2secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.2
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = MHA_LAS(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    # lr_scheduler="CyclicLR"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="mha_las",
    sub_dir="2secs",
    version=0
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    precision=16,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm2,
    ckpt_path="checkpoints/MHA_LAS/2secs/last.ckpt"
)

## Train (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.2
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = MHA_LAS(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    # lr_scheduler="CyclicLR"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="mha_las",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    precision=16,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm3,
    ckpt_path="checkpoints/MHA_LAS/3secs/last.ckpt"
)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15,
    num_subcenters=3
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.2
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = MHA_LAS(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="mha_las",
    sub_dir="4secs",
    version=1
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    ckpt_path="checkpoints/MHA_LAS/4secs/epoch=28-step=26824.ckpt"
)

## Test (4 secs)

In [5]:
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.15,
    scale=20,
    num_subcenters=3
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.2
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = MHA_LAS(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="mha_las",
    sub_dir="4secs",
    version=1
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    num_sanity_val_steps=0
)
trainer.test(
    model, 
    dm4,
    ckpt_path="checkpoints/MHA_LAS/4secs/last.ckpt"
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\core\datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
Restoring states from the checkpoint path at checkpoints/MHA_LAS/4secs/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at checkpoints/MHA_LAS/4secs/last.ckpt


Testing: 100%|██████████| 84/84 [00:05<00:00, 18.94it/s]USA            0.667656
UK             0.181009
Australia      0.044510
Mexico         0.017804
Norway         0.017804
Spain          0.017804
Chile          0.011869
Italy          0.011869
Ireland        0.008902
Canada         0.008902
India          0.005935
Germany        0.002967
New Zealand    0.002967
dtype: float64
m    0.560831
f    0.439169
dtype: float64


Testing: 100%|██████████| 84/84 [00:23<00:00, 18.94it/s]

Computing scores: 100%|██████████| 10/10 [06:45<00:00, 40.51s/it]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.4932330846786499,
 'test_acc_epoch': 0.4932330846786499,
 'test_f1': 0.45403939485549927,
 'test_f1_epoch': 0.45403939485549927,
 'test_top1_acc': 0.4932330846786499,
 'test_top1_acc_epoch': 0.4932330846786499,
 'test_top5_acc': 0.6526316404342651,
 'test_top5_acc_epoch': 0.6526316404342651}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 84/84 [08:15<00:00,  5.90s/it]


D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\core\datamodule.py:469: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


[{'test_acc': 0.4932330846786499,
  'test_acc_epoch': 0.4932330846786499,
  'test_f1': 0.45403939485549927,
  'test_f1_epoch': 0.45403939485549927,
  'test_top1_acc': 0.4932330846786499,
  'test_top1_acc_epoch': 0.4932330846786499,
  'test_top5_acc': 0.6526316404342651,
  'test_top5_acc_epoch': 0.6526316404342651}]

# VAR_ECAPA

## Train (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/var_ecapa/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = Var_ECAPA(
    num_classes=num_classes,
    embeddings_dim=256, 
    loss_func=aams
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="var_ecapa",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/var_ecapa/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = Var_ECAPA(
    num_classes=num_classes,
    embeddings_dim=256, 
    loss_func=aams
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="var_ecapa",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm4)

# ResNetSE

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnetse/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
num_filters = [16, 32, 64, 128]
model = ResNetSE(
    SEBasicBlock,
    [3, 4, 6, 3],
    num_filters,
    nOut=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="resnetse",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

# ResNetSEV2

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnetsev2/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
num_filters = [32, 64, 128, 256]
model = ResNetSEV2(
    SEBasicBlock,
    [3, 4, 6, 3],
    num_filters,
    nOut=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="resnetsev2",
    sub_dir="3secs",
    version=0
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm3,
    ckpt_path="checkpoints/resnetsev2/3secs/last.ckpt"
)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnetsev2/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
num_filters = [32, 64, 128, 256]
model = ResNetSEV2(
    SEBasicBlock,
    [3, 4, 6, 3],
    num_filters,
    nOut=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="resnetsev2",
    sub_dir="4secs",
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    # ckpt_path="checkpoints/resnetsev2/3secs/last.ckpt"
)

# ResNet20

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet20/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet20(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    # lr_scheduler="CyclicLR",
    # lr_scheduler_interval="step"
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet20",
    sub_dir="4secs"
)
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    # ckpt_path="checkpoints/resnet34se/3secs/last.ckpt"
)